# Sumário

1. [Pré-Processamento](Pré)
2. [Machine Learning](ml)
3. [Gráficos](gf)



# Pré-Processamento dados Reais


In [33]:
import pandas as pd
import os
import numpy as np


In [34]:

df1 = pd.read_csv('https://raw.githubusercontent.com/GUIPETAV/Base/main/df1.csv')# falha de subcarga
df2 = pd.read_csv('https://raw.githubusercontent.com/GUIPETAV/Base/main/df2.csv')# sem falha de subcarga


In [35]:
df10 = pd.read_csv('https://raw.githubusercontent.com/GUIPETAV/Base/main/df10.csv') # falha de subcarga 10%
df20 = pd.read_csv('https://raw.githubusercontent.com/GUIPETAV/Base/main/df20.csv') # falha de subcarga 20%
df15 = pd.read_csv('https://raw.githubusercontent.com/GUIPETAV/Base/main/df15.csv') # falha de subcarga 15%
dfbs = pd.read_csv('https://raw.githubusercontent.com/GUIPETAV/Base/main/dfbs.csv') #  sem de falha de subcarga

In [36]:
import pandas as pd


# Obter os nomes das colunas de ambos os DataFrames
columns_df2 = set(df2.columns)
columns_dfbs = set(dfbs.columns)

# Encontrar as colunas com nomes diferentes
different_columns_df2 = columns_df2.difference(columns_dfbs)
different_columns_dfbs = columns_dfbs.difference(columns_df2)

# Criar listas vazias para as colunas do DataFrame
different_columns_df2_list = list(different_columns_df2)
different_columns_dfbs_list = list(different_columns_dfbs)

# Preencher com None para manter as listas com o mesmo tamanho
max_length = max(len(different_columns_df2_list), len(different_columns_dfbs_list))
different_columns_df2_list.extend([None] * (max_length - len(different_columns_df2_list)))
different_columns_dfbs_list.extend([None] * (max_length - len(different_columns_dfbs_list)))

# Criar um DataFrame para a tabela
table_data = {
    'Colunas do df2 com nomes diferentes do dfbs': different_columns_df2_list,
    'Colunas do dfbs com nomes diferentes do df2': different_columns_dfbs_list
}

result_table = pd.DataFrame(table_data)


result_table

,Colunas do df2 com nomes diferentes do dfbs,Colunas do dfbs com nomes diferentes do df2
0,RTU_REFG_SUCT_TEMP_2,RTU_SEN_CAPA
1,RTU_REFG_DISC_PRES_1,RTU_REFG_DISC_PRES
2,time,RTU_REFG_SUCT_TEMP
3,RTU_REFG_SUCT_TEMP_1,RTU_TOT_CAPA
4,date,RTU_RA_FLOW
5,RTU_REFG_COND_TEMP_2,RTU_REFG_DISC_TEMP
6,RTU_REFG_COND_TEMP_1,RTU_STG_STA
7,RTU_REFG_SUCT_PRES_1,RTU_REFG_COND_PRES
8,RTU_REFG_SUCT_PRES_2,RTU_REFG_SUCT_PRES
9,RTU_REFG_DISC_TEMP_1,RTU_REFG_COND_TEMP


In [37]:
# Remover as features incongruentes

columns_to_drop_real = ['Unnamed: 0','time','date','month'
    'RTU_MA_HUM', 'RTU_MA_TEMP', 'RTU_LA_COND_TEMP',]

In [38]:
df1_t = df1.assign(Target = 1) # falha de subcarga

df2_t = df2.assign(Target = 0) # não falha de subcarga



In [39]:
# Eliminando  os momentos de desligameno do sistema no  site 2 de forma continua
# Seleciona as linhas que não contêm zero na FAN_WAAT e 'COMP_WATT'.

df1_c = df1_t.loc[(df1_t['RTU_SA_FAN_WATT'] != 0) & (df1_t['RTU_COMP_WATT'] != 0)]
df2_c = df2_t.loc[(df2_t['RTU_SA_FAN_WATT'] != 0) & (df2_t['RTU_COMP_WATT'] != 0)]

In [40]:
# Cria uma nova feature com a soma das pressões
df1_c = df1_c.assign(
    RTU_REFG_DISC_PRES=df1_c['RTU_REFG_DISC_PRES_1'] + df1_c['RTU_REFG_DISC_PRES_2'],
    RTU_REFG_SUCT_PRES=df1_c['RTU_REFG_SUCT_PRES_1'] + df1_c['RTU_REFG_SUCT_PRES_2'])

df2_c = df2_c.assign(
    RTU_REFG_DISC_PRES=df2_c['RTU_REFG_DISC_PRES_1'] + df2_c['RTU_REFG_DISC_PRES_2'],
    RTU_REFG_SUCT_PRES=df2_c['RTU_REFG_SUCT_PRES_1'] + df2_c['RTU_REFG_SUCT_PRES_2'])


#### Padronizar a Pressão 

In [41]:
# Passo 1: Calcular a Derivada da Pressão (N-1) por Minuto

# Certifique-se de que as colunas de data/hora estão no formato de data/hora do pandas
df1_c['Datetime'] = pd.to_datetime(df1_c['Datetime'])

df2_c['Datetime'] = pd.to_datetime(df2_c['Datetime'])

# Ordene o DataFrame pela coluna 'DataHora' (caso não esteja ordenado)
df1_c.sort_values(by='Datetime', inplace=True)

df2_c.sort_values(by='Datetime', inplace=True)


# Calcule a derivada por minuto para 'RTU_REFG_DISC_PRES'

df1_c['Derivada_RTU_REFG_DISC_PRES'] = df1_c['RTU_REFG_DISC_PRES'].diff() / df1_c['Datetime'].diff().dt.total_seconds()
df2_c['Derivada_RTU_REFG_DISC_PRES'] = df2_c['RTU_REFG_DISC_PRES'].diff() / df2_c['Datetime'].diff().dt.total_seconds()


# Calcule a derivada por minuto para 'RTU_REFG_SUCT_PRES'
df1_c['Derivada_RTU_REFG_SUCT_PRES'] = df1_c['RTU_REFG_SUCT_PRES'].diff() / df1_c['Datetime'].diff().dt.total_seconds()

df2_c['Derivada_RTU_REFG_SUCT_PRES'] = df2_c['RTU_REFG_SUCT_PRES'].diff() / df2_c['Datetime'].diff().dt.total_seconds()



In [42]:
# Passo 2 Calcular a Média móvel 


janela_media_movel = 1440   # Janela de um dia (1440 minutos)

# Calcule a média móvel da derivada por minuto para 'RTU_REFG_DISC_PRES'
df1_c['MediaMovel_Derivada_RTU_REFG_DISC_PRES'] = df1_c['Derivada_RTU_REFG_DISC_PRES'].rolling(window=janela_media_movel).mean()

# Calcule a média móvel da derivada por minuto para 'RTU_REFG_SUCT_PRES'
df1_c['MediaMovel_Derivada_RTU_REFG_SUCT_PRES'] = df1_c['Derivada_RTU_REFG_SUCT_PRES'].rolling(window=janela_media_movel).mean()

# Calcule a média móvel da derivada por minuto para 'RTU_REFG_DISC_PRES'
df2_c['MediaMovel_Derivada_RTU_REFG_DISC_PRES'] = df2_c['Derivada_RTU_REFG_DISC_PRES'].rolling(window=janela_media_movel).mean()

# Calcule a média móvel da derivada por minuto para 'RTU_REFG_SUCT_PRES'
df2_c['MediaMovel_Derivada_RTU_REFG_SUCT_PRES'] = df2_c['Derivada_RTU_REFG_SUCT_PRES'].rolling(window=janela_media_movel).mean()

In [43]:
# Passo 3 Calcular o Desvio Padrão


janela_um_dia = 1440 # Defina a janela de 1 dia (1440 minutos)


# Calcule o desvio padrão da derivada por minuto dentro da janela de 1 dia para 'RTU_REFG_DISC_PRES'
df1_c['DesvioPadrao_Derivada_RTU_REFG_DISC_PRES'] = df1_c['Derivada_RTU_REFG_DISC_PRES'].rolling(window=janela_um_dia).std()

# Calcule o desvio padrão da derivada por minuto dentro da janela de 1 dia para 'RTU_REFG_SUCT_PRES'
df1_c['DesvioPadrao_Derivada_RTU_REFG_SUCT_PRES'] = df1_c['Derivada_RTU_REFG_SUCT_PRES'].rolling(window=janela_um_dia).std()

# Calcule o desvio padrão da derivada por minuto dentro da janela de 1 dia para 'RTU_REFG_DISC_PRES'
df2_c['DesvioPadrao_Derivada_RTU_REFG_DISC_PRES'] = df2_c['Derivada_RTU_REFG_DISC_PRES'].rolling(window=janela_um_dia).std()

# Calcule o desvio padrão da derivada por minuto dentro da janela de 1 dia para 'RTU_REFG_SUCT_PRES'
df2_c['DesvioPadrao_Derivada_RTU_REFG_SUCT_PRES'] = df2_c['Derivada_RTU_REFG_SUCT_PRES'].rolling(window=janela_um_dia).std()



In [44]:
# Passo 4 - Padronizar Manualmente

# Calcule o Z-Score para cada minuto usando a média e o desvio padrão das colunas
df1_c['Z_Score_Derivada_RTU_REFG_SUCT_PRES'] = (df1_c['Derivada_RTU_REFG_SUCT_PRES'] - df1_c['MediaMovel_Derivada_RTU_REFG_SUCT_PRES']) / df1_c['DesvioPadrao_Derivada_RTU_REFG_SUCT_PRES']

df1_c['Z_Score_Derivada_RTU_REFG_DISC_PRES'] = (df1_c['Derivada_RTU_REFG_DISC_PRES'] - df1_c['MediaMovel_Derivada_RTU_REFG_DISC_PRES']) / df1_c['DesvioPadrao_Derivada_RTU_REFG_DISC_PRES']

df2_c['Z_Score_Derivada_RTU_REFG_SUCT_PRES'] = (df2_c['Derivada_RTU_REFG_SUCT_PRES'] - df2_c['MediaMovel_Derivada_RTU_REFG_SUCT_PRES']) / df2_c['DesvioPadrao_Derivada_RTU_REFG_SUCT_PRES']

df2_c['Z_Score_Derivada_RTU_REFG_DISC_PRES'] = (df2_c['Derivada_RTU_REFG_DISC_PRES'] - df2_c['MediaMovel_Derivada_RTU_REFG_DISC_PRES']) / df2_c['DesvioPadrao_Derivada_RTU_REFG_DISC_PRES']

In [45]:
df2_c.tail()

,Unnamed: 0,Datetime,date,time,month,RTU_COMP_WATT,RTU_LA_COND_TEMP,RTU_MA_HUM,RTU_MA_TEMP,RTU_OA_HUM,...,RTU_REFG_DISC_PRES,RTU_REFG_SUCT_PRES,Derivada_RTU_REFG_DISC_PRES,Derivada_RTU_REFG_SUCT_PRES,MediaMovel_Derivada_RTU_REFG_DISC_PRES,MediaMovel_Derivada_RTU_REFG_SUCT_PRES,DesvioPadrao_Derivada_RTU_REFG_DISC_PRES,DesvioPadrao_Derivada_RTU_REFG_SUCT_PRES,Z_Score_Derivada_RTU_REFG_SUCT_PRES,Z_Score_Derivada_RTU_REFG_DISC_PRES
258579,258579,2021-09-29 16:58:00,2021-09-29,16:58:00,9,255.000015,18.664831,56.994507,19.833544,63.551861,...,15.702408,15.802378,-0.007623,0.011428,-0.003095,0.001649,0.029066,0.027648,0.353729,-0.155764
258580,258580,2021-09-29 16:59:00,2021-09-29,16:59:00,9,1983.000122,20.207827,56.758289,19.832773,62.563267,...,17.758046,13.318046,0.034261,-0.041406,-0.003072,0.001620,0.029082,0.027671,-1.554894,1.283675
258581,258581,2021-09-29 17:00:00,2021-09-29,17:00:00,9,445.000031,22.923174,57.022457,19.856422,62.345287,...,15.447086,14.390031,-0.038516,0.017866,-0.003099,0.001632,0.029097,0.027674,0.586623,-1.217202
258582,258582,2021-09-29 17:01:00,2021-09-29,17:01:00,9,3532.000000,22.515036,57.199764,19.847624,63.332333,...,18.055580,13.126932,0.043475,-0.021052,-0.003069,0.001617,0.029123,0.027681,-0.818953,1.598178
258583,258583,2021-09-29 17:02:00,2021-09-29,17:02:00,9,2766.000000,23.849806,57.171261,19.763884,62.924618,...,17.939455,10.044896,-0.001935,-0.051367,-0.003070,0.001582,0.029123,0.027716,-1.910436,0.038970


In [46]:
# Elimina as features
columns_to_drop = ['Unnamed: 0',
    'RTU_MA_HUM', 'RTU_MA_TEMP', 'RTU_LA_COND_TEMP',
    'RTU_REFG_DISC_PRES_1', 'RTU_REFG_DISC_PRES_2',
    'RTU_REFG_SUCT_PRES_1', 'RTU_REFG_SUCT_PRES_2',
    'RTU_REFG_COND_TEMP_1', 'RTU_REFG_COND_TEMP_2',
    'RTU_REFG_SUCT_TEMP_1', 'RTU_REFG_SUCT_TEMP_2',
    'RTU_REFG_DISC_TEMP_1', 'RTU_REFG_DISC_TEMP_2',
]

df1_x = df1_c.drop(columns_to_drop, axis=1)
df2_x = df2_c.drop(columns_to_drop, axis=1)

In [47]:
# Converter fahrenheit para celsius
def fahrenheit_to_celsius(fahrenheit):
    celsius = (fahrenheit - 32) * 5/9
    return celsius

# Colunas a serem convertidas
columns_to_convert = ['RTU_RA_TEMP', 'RTU_OA_TEMP', 'ZA_TEMP', 'RTU_REFG_COND_TEMP',
                      'RTU_REFG_SUCT_TEMP', 'RTU_REFG_DISC_TEMP','RTU_SA_TEMP']


# Aplicando a conversão para Celsius
df20[columns_to_convert] = df20[columns_to_convert].apply(fahrenheit_to_celsius)
dfbs[columns_to_convert] = dfbs[columns_to_convert].apply(fahrenheit_to_celsius)

In [48]:
# Converter colunas de Pascal para Bar
df20['RTU_REFG_SUCT_PRES'], df20['RTU_REFG_DISC_PRES'] = df20['RTU_REFG_SUCT_PRES'] /100000 , df20['RTU_REFG_DISC_PRES'] /100000
dfbs['RTU_REFG_SUCT_PRES'], dfbs['RTU_REFG_DISC_PRES'] = dfbs['RTU_REFG_SUCT_PRES'] / 100000, dfbs['RTU_REFG_DISC_PRES'] /100000

In [49]:

df20_t = df20.assign(Target = 1) # falha de subcarga

dfbs_t = dfbs.assign(Target = 0) #  não falha

In [50]:
# Eliminando  os momentos de desligameno do sistema no  site 2 de forma continua
# Seleciona as linhas que não contêm zero na FAN_WAAT e 'COMP_WATT'.

df20_c = df20_t.loc[(df20_t['RTU_COMP_WATT'] != 0) ]
dfbs_c = dfbs_t.loc[(dfbs_t['RTU_COMP_WATT'] != 0) ]

In [51]:
columns_to_drop = ['Unnamed: 0',
    'RTU_RA_FLOW', 'RTU_OA_FLOW', 'RTU_STG_STA',
    'RTU_REFG_COND_PRES', 'RTU_SEN_CAPA',
    'RTU_TOT_CAPA', 'ZA_TEMP_SPT',
]

df20_c = df20_c.drop(columns_to_drop, axis=1)
dfbs_c = dfbs_c.drop(columns_to_drop, axis=1)


In [52]:
# Definir y real

df1_y= df1_c['Target']
df2_y= df2_c['Target']

In [53]:
# # Definir X real
# df1_x= df1_x.drop(['Datetime','Target', 'RTU_REFG_COND_TEMP', 'RTU_REFG_SUCT_TEMP', 'RTU_REFG_DISC_TEMP'], axis=1)
# df2_x= df2_x.drop(['Datetime','Target', 'RTU_REFG_COND_TEMP', 'RTU_REFG_SUCT_TEMP', 'RTU_REFG_DISC_TEMP'], axis=1)


In [54]:
# Definir X real sem features temp modificadas
df1_x= df1_x.drop(['Datetime','Target','RTU_RA_TEMP','RTU_OA_TEMP','ZA_TEMP', 'RTU_REFG_COND_TEMP', 'RTU_REFG_SUCT_TEMP', 'RTU_REFG_DISC_TEMP','RTU_SA_TEMP'], axis=1)
df2_x= df2_x.drop(['Datetime','Target','RTU_RA_TEMP','RTU_OA_TEMP','ZA_TEMP', 'RTU_REFG_COND_TEMP', 'RTU_REFG_SUCT_TEMP', 'RTU_REFG_DISC_TEMP','RTU_SA_TEMP'], axis=1)

KeyError: "['RTU_REFG_COND_TEMP', 'RTU_REFG_SUCT_TEMP', 'RTU_REFG_DISC_TEMP'] not found in axis"

In [ ]:
# Definir X sim

# df20_x= df20_c.drop(['Datetime','Target', 'RTU_REFG_COND_TEMP', 'RTU_REFG_SUCT_TEMP', 'RTU_REFG_DISC_TEMP'], axis=1)
# dfbs_x= dfbs_c.drop(['Datetime','Target', 'RTU_REFG_COND_TEMP', 'RTU_REFG_SUCT_TEMP', 'RTU_REFG_DISC_TEMP'], axis=1)

In [ ]:
# Definir X sim sem features temp modificadas

df20_x= df20_c.drop(['Datetime','Target','RTU_RA_TEMP','RTU_OA_TEMP','ZA_TEMP', 'RTU_REFG_COND_TEMP', 'RTU_REFG_SUCT_TEMP', 'RTU_REFG_DISC_TEMP','RTU_SA_TEMP'], axis=1)
dfbs_x= dfbs_c.drop(['Datetime','Target','RTU_RA_TEMP','RTU_OA_TEMP','ZA_TEMP', 'RTU_REFG_COND_TEMP', 'RTU_REFG_SUCT_TEMP', 'RTU_REFG_DISC_TEMP','RTU_SA_TEMP'], axis=1)

In [ ]:
# Definir y sim

df20_y = df20_c['Target']
dfbs_y = dfbs_c['Target']
